In [103]:
#Imports of libraries, pandas options

import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

pd.options.mode.chained_assignment = None  # default='warn', to avoid pd error when editing with for loop, there's no for loops as of now, but it might be needed later

In [104]:
#Creation of marketwatch data links

marketwatchdata = {}

ticker=input("Insert symbol to research: ")
#For stocks on NYSE or NASDAQ just write the ticker
#For stocks on other exchanges Marketwatch does not always have data 
#If you want to find the data they might have, use 'ticker'?countrycode='countrycode'

#
urlincome = 'https://www.marketwatch.com/investing/stock/' + ticker + '/financials'
urlbalance = 'https://www.marketwatch.com/investing/stock/' + ticker + '/financials/balance-sheet'
urlcashflow = 'https://www.marketwatch.com/investing/stock/' + ticker + '/financials/cash-flow'
urlkeydata = 'https://www.marketwatch.com/investing/stock/' + ticker

Insert symbol to research: O


In [105]:
#Scrape of marketwatch data, not a perfect source but will do for this program

income = pd.read_html(urlincome)
balancesheet = pd.read_html(urlbalance)
cashflow = pd.read_html(urlcashflow)
keydata = pd.read_html(urlkeydata)

In [106]:
#general formatting

income_statement = income[4].rename(columns={'Item  Item': 'Income statement'})

balance_sheet_assets = balancesheet[4].rename(columns={'Item  Item': 'Assets'})
balance_sheet_liabilities = balancesheet[5].rename(columns={'Item  Item': 'Liabilities and Equity'})

cashflow_operations = cashflow[4].rename(columns={'Item  Item': 'Cash-flow from Operating activities'})
cashflow_investing = cashflow[5].rename(columns={'Item  Item': 'Cash-flow from Investing activities'})
cashflow_financing = cashflow[6].rename(columns={'Item  Item': 'Cash-flow from Financing activities'})

dayprice = keydata[1].rename(columns={'Close': 'Stock price'})
performance = keydata[4].rename(columns={'0': 'Performance'})
competitors = keydata[5].rename(columns={'Name': 'Competitors'})

In [111]:
cashflow_operations

,Cash-flow from Operating activities,2015,2016,2017,2018,2019,5-year trend
0,Funds from Operations Funds fro...,=688.78*1000,=770.81*1000,=849.85*1000,=928.95*1000,=1.05*1000000,NaN
1,Funds from Operations Growth Fu...,=0,=11.91%,=10.25%,=9.31%,=13.44%,NaN
2,Changes in Working Capital Chan...,=3.53*1000,=33.24*1000,=26*1000,=11.79*1000,=15.1*1000,NaN
3,Receivables Receivables,=0,=0,=0,=0,=0,NaN
4,Inventories Inventories,=0,=0,=0,=0,=0,NaN
5,Accounts Payable Accounts Payable,=6.17*1000,=34.47*1000,=26.1*1000,=18.7*1000,=24.06*1000,NaN
6,Other Assets/Liabilities Other ...,=(2.64*1000),=(1.23*1000),=(92),=(6.9*1000),=(8.95*1000),NaN
7,Net Operating Cash Flow Net Ope...,=692.3*1000,=804.05*1000,=875.85*1000,=940.74*1000,=1.07*1000000,NaN
8,Net Operating Cash Flow Growth ...,=0,=16.14%,=8.93%,=7.41%,=13.63%,NaN


In [108]:
#after eons and eons, I was finally able to convert the B to billions, M to millions and K to thousands, efficiently
#conversion of income statement
income_statement.iloc[:, [1,2,3,4,5]] = income_statement.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)

income_statement.iloc[:, [1,2,3,4,5]] = '=' + income_statement.iloc[:, [1,2,3,4,5]].astype(str)

#conversion of balance sheet
balance_sheet_assets.iloc[:, [1,2,3,4,5]] = balance_sheet_assets.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)
balance_sheet_liabilities.iloc[:, [1,2,3,4,5]] = balance_sheet_liabilities.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)

balance_sheet_assets.iloc[:, [1,2,3,4,5]] = '=' + balance_sheet_assets.iloc[:, [1,2,3,4,5]].astype(str)
balance_sheet_liabilities.iloc[:, [1,2,3,4,5]] = '=' + balance_sheet_liabilities.iloc[:, [1,2,3,4,5]].astype(str)

#conversion of cashflow statement
cashflow_operations.iloc[:, [1,2,3,4,5]] = cashflow_operations.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)
cashflow_investing.iloc[:, [1,2,3,4,5]] = cashflow_investing.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)
cashflow_financing.iloc[:, [1,2,3,4,5]] = cashflow_financing.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)


cashflow_operations.iloc[:, [1,2,3,4,5]] = '=' + cashflow_operations.iloc[:, [1,2,3,4,5]].astype(str)
cashflow_investing.iloc[:, [1,2,3,4,5]] = '=' + cashflow_investing.iloc[:, [1,2,3,4,5]].astype(str)
cashflow_financing.iloc[:, [1,2,3,4,5]] = '=' + cashflow_financing.iloc[:, [1,2,3,4,5]].astype(str)

#General data gets converted to num, to facilitate excel use

competitors.iloc[:,[2]] = competitors.iloc[:,[2]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0', '\$':'='}, regex=True)
dayprice.iloc[:,[0]] = dayprice.iloc[:,[0]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0', '\$':'='}, regex=True)

In [110]:
income_statement.iloc[:, [0]] = income_statement.iloc[:, [0]].apply(lambda x: x.str.slice(0, 36))

cashflow_operations.iloc[:, [0]] = cashflow_operations.iloc[:, [0]].apply(lambda x: x.str.slice(0, 39))
cashflow_investing.iloc[:, [0]] = cashflow_investing.iloc[:, [0]].apply(lambda x: x.str.slice(0, 36))
cashflow_financing.iloc[:, [0]] = cashflow_financing.iloc[:, [0]].apply(lambda x: x.str.slice(0, 38))

balance_sheet_assets.iloc[:, [0]] = balance_sheet_assets.iloc[:, [0]].apply(lambda x: x.str.slice(0, 37))
balance_sheet_liabilities.iloc[:, [0]] = balance_sheet_liabilities.iloc[:, [0]].apply(lambda x: x.str.slice(0, 61))

In [114]:
#Saving model to Excel
try:
    datawriter = pd.ExcelWriter(ticker + '.xlsx', engine='xlsxwriter')   #xlsx automatically changes to ticker.xlsx
except:
    datawriter = pd.ExcelWriter('somethingwentwrong.xlsx', engine='xlsxwriter') #don't put weird stuff as ticker, there's no error handler
#title1 = {' key stock data': str(ticker)}
#title2 = {' three statement model': str(ticker)}

#formatting
cell_format = workbook.add_format({'font_family': 'arial'})

#title1.to_excel(datawriter, sheet_name='Key data', startrow=0, startcol=1)
dayprice.to_excel(datawriter, sheet_name='Key data',startrow=2, startcol=1)
performance.to_excel(datawriter, sheet_name='Key data',startrow=5, startcol=1)
competitors.to_excel(datawriter, sheet_name='Key data',startrow=2, startcol=6)

#title2.to_excel(datawriter, sheet_name='Analysis', startrow=0, startcol=1)
income_statement.to_excel(datawriter, sheet_name='Analysis',startrow=2, startcol=1)
balance_sheet_assets.to_excel(datawriter, sheet_name='Analysis',startrow=2, startcol=9)
balance_sheet_liabilities.to_excel(datawriter, sheet_name='Analysis',startrow=42, startcol=9)

cashflow_operations.to_excel(datawriter, sheet_name='Analysis',startrow=2, startcol=17)
cashflow_investing.to_excel(datawriter, sheet_name='Analysis',startrow=24, startcol=17)
cashflow_financing.to_excel(datawriter, sheet_name='Analysis',startrow=43, startcol=17)

In [115]:
datawriter.save()